In [1]:
import numpy as np
import random
import pandas as pd
from sklearn.linear_model import LogisticRegression

In [2]:
shopping_lists = []

user_ids = list(range(10))
qty = list(range(1,11))

items = {'meats' : 0 , 'frozen': 1, 'drinks' : 2,'Snacks' : 3 , 'icecream' : 4,'hard_liquor' : 5,'fruits':6}
prices = {'meats' : 10, 'frozen' : 5, 'drinks': 10, 'Snacks' : 5, 'icecream' : 6, 'hard_liquor' : 12,'fruits': 5}

num_users = len(user_ids)
num_items = len(items)
shopping_lists = []
recommendation_matrix = np.zeros((num_users,num_items))

### Cocurrence Matrix for recommendations 

In [3]:
for _ in range(100000):
    shopping_list = {}
    happy_probabilty = 0.5
    shopping_list['user_id'] = random.sample(user_ids,1)[0]
    shopping_items = random.sample(list(items.keys()),3)
    
    shopping_list['prices'] = []
    shopping_list['happy'] = 0
    for item in items:
        shopping_list['items_' + item] = 0
    
    for item in shopping_items:
        shopping_list['prices'].append(prices[item])
        shopping_list['items_'+ item] = 1
        if item == 'frozen':
            happy_probabilty -= 0.1
        if item == 'hard_liquor':
            happy_probabilty -= 0.1
       
        if item == 'icecream':
            happy_probabilty += 0.2
                    
        if item == 'fruit':
            happy_probabilty += 0.1
        
        #print(shopping_list['user_id'],items[item])
        recommendation_matrix[shopping_list['user_id']][items[item]] += 1

            
    shopping_list['qty'] = random.sample(qty,3)
    if random.random() <= happy_probabilty:
        shopping_list['happy'] = 1
   
    shopping_lists.append(shopping_list)
    
print (recommendation_matrix) 

[[ 4130.  4116.  4363.  4292.  4326.  4170.  4315.]
 [ 4255.  4178.  4266.  4239.  4285.  4313.  4296.]
 [ 4274.  4277.  4271.  4239.  4411.  4368.  4388.]
 [ 4164.  4379.  4419.  4276.  4266.  4364.  4276.]
 [ 4302.  4229.  4218.  4361.  4356.  4263.  4406.]
 [ 4451.  4394.  4426.  4401.  4362.  4302.  4309.]
 [ 4166.  4212.  4230.  4269.  4188.  4314.  4198.]
 [ 4240.  4325.  4339.  4294.  4174.  4376.  4339.]
 [ 4273.  4281.  4232.  4272.  4266.  4343.  4219.]
 [ 4263.  4266.  4234.  4275.  4244.  4247.  4225.]]


### Now for an existing user

In [4]:
#say user 0
print (recommendation_matrix[0]) 

# we will predict the maximum value as product
print (np.argmax(recommendation_matrix[0])) #so it is Drinks

[ 4130.  4116.  4363.  4292.  4326.  4170.  4315.]
2


In [5]:
for li in shopping_lists[:10]:
    print(li)

{'prices': [6, 5, 10], 'happy': 1, 'items_hard_liquor': 0, 'user_id': 5, 'items_meats': 0, 'items_icecream': 1, 'items_Snacks': 1, 'items_fruits': 0, 'qty': [4, 2, 10], 'items_frozen': 0, 'items_drinks': 1}
{'prices': [5, 12, 6], 'happy': 0, 'items_hard_liquor': 1, 'user_id': 7, 'items_meats': 0, 'items_icecream': 1, 'items_Snacks': 1, 'items_fruits': 0, 'qty': [9, 5, 8], 'items_frozen': 0, 'items_drinks': 0}
{'prices': [10, 5, 12], 'happy': 0, 'items_hard_liquor': 1, 'user_id': 6, 'items_meats': 1, 'items_icecream': 0, 'items_Snacks': 0, 'items_fruits': 1, 'qty': [1, 3, 4], 'items_frozen': 0, 'items_drinks': 0}
{'prices': [5, 10, 6], 'happy': 1, 'items_hard_liquor': 0, 'user_id': 0, 'items_meats': 1, 'items_icecream': 1, 'items_Snacks': 0, 'items_fruits': 1, 'qty': [1, 2, 4], 'items_frozen': 0, 'items_drinks': 0}
{'prices': [5, 5, 10], 'happy': 0, 'items_hard_liquor': 0, 'user_id': 9, 'items_meats': 0, 'items_icecream': 0, 'items_Snacks': 0, 'items_fruits': 1, 'qty': [1, 3, 9], 'items

In [6]:
a = pd.DataFrame.from_dict(shopping_lists)
columns_of_interest = a.columns[1:-3]
print (columns_of_interest)

Index(['items_Snacks', 'items_drinks', 'items_frozen', 'items_fruits',
       'items_hard_liquor', 'items_icecream', 'items_meats'],
      dtype='object')


In [7]:
training_data = a.iloc[:80000]
testing_data = a.iloc[80000:]
regressor = LogisticRegression()
regressor.fit(training_data[columns_of_interest],training_data['happy'])
testing_predicted_labels = regressor.predict(testing_data[columns_of_interest])
testing_real_labels = testing_data['happy'].as_matrix()

In [9]:
regressor.predict_proba(testing_data[columns_of_interest]) #probabiltiy of the happy or not happy

array([[ 0.40347404,  0.59652596],
       [ 0.51270739,  0.48729261],
       [ 0.60046233,  0.39953767],
       ..., 
       [ 0.59610417,  0.40389583],
       [ 0.60362293,  0.39637707],
       [ 0.49360417,  0.50639583]])

In [10]:
accuracy = np.sum(testing_real_labels == testing_predicted_labels)/len(testing_real_labels)
print (accuracy)

0.6153
